# Generating object masks from input prompts with SAM

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/input_prompts.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/segment-geospatial&urlpath=lab/tree/segment-geospatial/docs/examples/input_prompts.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/input_prompts.ipynb)

This notebook shows how to generate object masks from input prompts with the Segment Anything Model (SAM).

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

The notebook is adapted from [segment-anything/notebooks/predictor_example.ipynb](https://github.com/opengeos/segment-anything/blob/pypi/notebooks/predictor_example.ipynb), but I have made it much easier to save the segmentation results and visualize them.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [4]:
#%pip install segment-geospatial
#%pip install leafmap
#%pip install rioxarray

In [5]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff

In [15]:
url = "https://drive.google.com/file/d/11-wM9mPONwvFaUycktDwnWAdXM5kwH4A/view?usp=sharing"
leafmap.download_file(url, output="image.tif")

image.tif already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/image.tif'

In [9]:
import rioxarray as rxr
img_raster=rxr.open_rasterio("/content/image.tif")
print("The crs of your data is:", img_raster.rio.crs)
print("The nodatavalue of your data is:", img_raster.rio.nodata)
print("The shape of your data is:", img_raster.shape)
print("The spatial resolution for your data is:", img_raster.rio.resolution())
print("The metadata for your data is:", img_raster.attrs)
print("extent", img_raster.rio.bounds() )

The crs of your data is: EPSG:26911
The nodatavalue of your data is: 255
The shape of your data is: (4, 471, 1413)
The spatial resolution for your data is: (0.007128879999976225, -0.00712887999935831)
The metadata for your data is: {'AREA_OR_POINT': 'Area', 'DataType': 'Generic', 'RepresentationType': 'ATHEMATIC', '_FillValue': 255, 'scale_factor': 1.0, 'add_offset': 0.0}
extent (523281.39023086417, 4779516.929034361, 523291.46333830414, 4779520.286736839)


In [16]:
m = leafmap.Map(#height="600px"
                )
#m.add_basemap("SATELLITE")
m.add_raster("image.tif", layer_name="Image")
m.add_layer_manager()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Create an interactive map

In [ ]:
m = leafmap.Map(center=[37.6412, -122.1353], zoom=15, height="800px")
m.add_basemap("SATELLITE")
m

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [18]:
if m.user_roi is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [523281.39023086417, 4779516.929034361, 523291.46333830414, 4779520.286736839] #[-122.1497, 37.6311, -122.1203, 37.6458]

In [21]:
image = "/content/image.tif" #"satellite.tif"
#tms_to_geotiff(output=image,bbox=bbox,zoom=20,
               # source="Satellite",              overwrite=True)

You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [22]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=1110.0, center=[35.17380831799959, -164.267578125], controls=(ZoomControl(options=['position', 'zoo…

## Initialize SAM class

Specify the file path to the model checkpoint. If it is not specified, the model will to downloaded to the working directory.

Set `automatic=False` to disable the `SamAutomaticMaskGenerator` and enable the `SamPredictor`.

In [23]:
sam = SamGeo(
    model_type="vit_h",
    automatic=False,
    sam_kwargs=None,
)

Model checkpoint for vit_h not found.


Downloading...
From: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
To: /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth
100%|██████████| 2.56G/2.56G [01:37<00:00, 26.4MB/s]


Specify the image to segment.

In [ ]:
sam.set_image(image)

## Image segmentation with input points

A single point can be used to segment an object. The point can be specified as a tuple of (x, y), such as (col, row) or (lon, lat). The points can also be specified as a file path to a vector dataset. For non (col, row) input points, specify the `point_crs` parameter, which will automatically transform the points to the image column and row coordinates.

Try a single point input:

In [25]:
point_coords = [[ 523290,  4779518 ]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:26911", output="mask1.tif")
m.add_raster("mask1.tif", layer_name="Mask1", nodata=0, cmap="Blues", opacity=1)
m

TypeError: invalid path or file: None

Try multiple points input:

In [ ]:
point_coords = [[-122.1464, 37.6431], [-122.1449, 37.6415], [-122.1451, 37.6395]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output="mask2.tif")
m.add_raster("mask2.tif", layer_name="Mask2", nodata=0, cmap="Greens", opacity=1)
m

## Interactive segmentation

Display the interactive map and use the marker tool to draw points on the map. Then click on the `Segment` button to segment the objects. The results will be added to the map automatically. Click on the `Reset` button to clear the points and the results.

In [26]:
m = sam.show_map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

![](https://i.imgur.com/2Nyg9uW.gif)